# Tests

In [ ]:
import torch

from _lib.w2v import run_w2v_tests


print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

### All hexes

In [5]:
resolution = 8
run_w2v_tests(resolution, f"optuna_w2v_{resolution}", GraphTypes.NONE)

[I 2022-05-12 10:08:23,147] Trial 55 finished with value: 0.4185958206653595 and parameters: {'optimizer': 'Adam', 'lr': 0.0003902911286761185, 'batch_size': 144, 'first_layer_size': 200, 'second_layer_size': 100}. Best is trial 28 with value: 0.41610187292099.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
wandb: wandb version 0.12.16 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
resolution = 9
run_w2v_tests(resolution, f"optuna_w2v_{resolution}", GraphTypes.NONE, n_trials=50, ds_split=0.1)

[I 2022-05-20 15:20:24,981] Trial 49 finished with value: 0.28854507207870483 and parameters: {'optimizer': 'Adam', 'lr': 0.0014063824765723897, 'batch_size': 512, 'first_layer_size': 175, 'second_layer_size': 100}. Best is trial 32 with value: 0.26827651262283325.


params: {'optimizer': 'Adam', 'lr': 0.0012138308896900158, 'batch_size': 496, 'first_layer_size': 200, 'second_layer_size': 125}
value: 0.26827651262283325


In [3]:
resolution = 10
run_w2v_tests(resolution, f"optuna_w2v_{resolution}", GraphTypes.NONE, n_trials=50, ds_split=0.01)

[I 2022-05-20 23:04:17,158] Trial 49 finished with value: 0.3387257754802704 and parameters: {'optimizer': 'Adagrad', 'lr': 0.0012975864540887905, 'batch_size': 320, 'first_layer_size': 200, 'second_layer_size': 100}. Best is trial 35 with value: 0.18402734398841858.


params: {'optimizer': 'Adagrad', 'lr': 0.029220995866855556, 'batch_size': 80, 'first_layer_size': 175, 'second_layer_size': 125}
value: 0.18402734398841858


### Hexes from itineraries

In [6]:
resolution = 8
run_w2v_tests(resolution, f"optuna_w2v_{resolution}_selected_hexes", GraphTypes.GRAPH)

[I 2022-05-14 16:58:52,101] Trial 99 finished with value: 0.4329105317592621 and parameters: {'optimizer': 'Adam', 'lr': 0.0003469784318374137, 'batch_size': 32, 'first_layer_size': 100, 'second_layer_size': 125}. Best is trial 95 with value: 0.4317620098590851.


params: {'optimizer': 'Adam', 'lr': 0.00037038763887552726, 'batch_size': 48, 'first_layer_size': 100, 'second_layer_size': 125}
value: 0.4317620098590851


In [7]:
resolution = 9
run_w2v_tests(resolution, f"optuna_w2v_{resolution}_selected_hexes", GraphTypes.GRAPH)

[I 2022-05-15 06:50:01,028] Trial 99 finished with value: 0.45207643508911133 and parameters: {'optimizer': 'Adagrad', 'lr': 0.005207445208280617, 'batch_size': 64, 'first_layer_size': 175, 'second_layer_size': 150}. Best is trial 86 with value: 0.4371359646320343.


params: {'optimizer': 'Adagrad', 'lr': 0.013431327409644152, 'batch_size': 112, 'first_layer_size': 175, 'second_layer_size': 150}
value: 0.4371359646320343


In [ ]:
resolution = 10
run_w2v_tests(resolution, f"optuna_w2v_{resolution}_selected_hexes", GraphTypes.GRAPH)

# Final training & vectors saving

In [3]:
import pandas as pd

from _lib.w2v import train_w2v_model, read_osm_data, save_vectors
from _lib.w2v import H3NeighborDataset, GraphTypes
from IPython.display import clear_output


final_vector_size = 50
params4final_training = pd.DataFrame.from_dict(
        {'8': [8, False, 64, 200, 150, 'Adam', 0.00025, 100],
        '8g': [8, True, 32, 100, 125, 'Adam', 0.00037, 100],
        '9': [9, False, 512, 200, 125, 'Adam', 0.001, 100],
        '9g': [9, True, 124, 175, 150, 'Adagrad', 0.01, 100],
        '10': [10, False, 64, 175, 125, 'Adagrad', 0.03, 100],
        '10g': [10, True, 512, 200, 150, 'Adam', 0.001, 100]}, 
    orient='index', 
    columns=['resolution', 'graph', 'batch_size', 'first_ls', 'second_ls', 'optim', 'lr', 'epochs'])

In [5]:
for i, params in params4final_training.iterrows():
    graph_type = GraphTypes.GRAPH if params.graph else GraphTypes.NONE
    df, graph = read_osm_data(params.resolution, graph_type)
    dataset = H3NeighborDataset(df, graph)
    model, trainer = train_w2v_model(f"w2v_final",
                                    dataset,
                                    params.epochs,
                                    params.batch_size,
                                    params.optim,
                                    params.lr,
                                    params.first_ls,
                                    params.second_ls,
                                    final_vector_size,
                                    p4wdb={'resolution': params.resolution, 'graph': params.graph})
    save_vectors(f"{params.resolution}{'_g' if params.graph else ''}", model, df)
    clear_output(wait=True)

100%|██████████| 214143/214143 [00:12<00:00, 16666.49it/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
